In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.colors as mcolors

from scipy import stats
from matplotlib import rcParams

from irt import Beta3
import matplotlib.pyplot as plt

In [ ]:
pij = pd.read_csv('./ReSys2_mrr.csv')
pij.set_index(pij.columns[0], inplace=True)


In [ ]:
normalized_df = pij

In [ ]:
def ICC_function(abilities, difficulties, discriminations):
    a = ((1-abilities)/ abilities)
    b = (difficulties / (1-difficulties))
    c = a*b
    d = c**discriminations
    return (1 / (d+1))

In [ ]:
b4 = Beta3(
        learning_rate=100, 
        epochs=10000,
        n_respondents=normalized_df.shape[1], 
        n_items=normalized_df.shape[0],
        n_workers=-1,
        random_seed=1,
    )
b4.fit(normalized_df.values)

In [ ]:
b4.abilities

In [ ]:
b4.difficulties

In [ ]:
b4.discriminations

In [ ]:
new_pij = pd.DataFrame(index=range(17), columns=range(11))

for i in range(17):
    for j in range(7):
        alpha = (b4.abilities[j] / b4.difficulties[i]) ** b4.discriminations[i]
        beta_val = ((1 - b4.abilities[j]) / (1 - b4.difficulties[i])) ** b4.discriminations[i]
        new_pij.iloc[i, j] = (alpha)/(alpha+beta_val)

In [ ]:
def plot_discriminations_difficulties(discriminations, difficulties, normalized_df, font_size=10, font_ann_size=8, base_point_size=500):
    import matplotlib.cm as cm
    
    rcParams['font.family'] = 'serif'
    rcParams['font.serif'] = ['Times New Roman']

    sns.set_style('whitegrid')
    fig, ax = plt.subplots(figsize=(3, 3))  # 调整图像大小
    ax.grid(color='black', linestyle='--', linewidth=0.5)
    
    ax.spines['bottom'].set_color('black')
    ax.spines['left'].set_color('black')
    ax.spines['right'].set_color('black')
    ax.spines['top'].set_color('black')
    ax.xaxis.label.set_color('black')
    ax.yaxis.label.set_color('black')
    ax.tick_params(axis='x', colors='black')
    ax.tick_params(axis='y', colors='black')
    
    # 定义颜色
    colors = cm.tab20.colors  # 使用 matplotlib 内置调色板 Tab10
    
    for i in range(len(discriminations)):
        ax.scatter(
            discriminations[i],
            difficulties[i],
            label=f'{normalized_df.index[i]}',
            edgecolors='black',  # 边框颜色
            facecolors=colors[i % len(colors)],  # 设置不同的颜色
            s=base_point_size,   # 控制点的大小
            marker='o'           # 使用相同的圆形 marker
        )
    
    plt.xlabel('Recommendation algorithm\nconsistency', fontsize=font_size, family='Times New Roman', color='black')
    plt.ylabel('Recommendation algorithm\ndifficulty limit', fontsize=font_size, family='Times New Roman', color='black')

    plt.legend(title='', fontsize=font_ann_size, bbox_to_anchor=(1, 0.75), loc='upper left', ncol=1, frameon=False)
    
    ax.set_ylim(-0.01, 0.3)
    ax.set_xlim(0, 3)
    
    # x_min = int(discriminations.min())
    # x_max = int(discriminations.max())
        
    # ax.set_xlim(x_min, x_max)
    plt.savefig("Figure4a.pdf", format="pdf", bbox_inches='tight')
    plt.show()

# 调用函数
plot_discriminations_difficulties(b4.discriminations, (1-b4.difficulties), normalized_df, font_size=10, font_ann_size=9, base_point_size=50)


In [ ]:
fairness_model = normalized_df.apply(np.mean,axis=0).to_numpy()

In [ ]:
def create_abilities_fairness_table(name, abilities, fairness_model):
    df = pd.DataFrame({
        'Model': name,
        'Ability': abilities,
        'STS': fairness_model
    })
    
    return df

df = create_abilities_fairness_table(pij.columns, 1-b4.abilities, fairness_model)
print(df)

In [ ]:
def f(theta,delta_j,a_j):
    term1 = (delta_j / (1 - delta_j)) ** a_j
    term2 = (theta / (1 - theta)) ** (-a_j - 1)
    numerator = a_j * term1 * term2
    denominator = (1 + term1 * (theta / (1 - theta)) ** -a_j) ** 2
    return numerator / denominator * (1 / (1 - theta) ** 2)

In [ ]:
abilities = np.linspace(0.001, 0.999, 1000)

In [ ]:
import matplotlib.cm as cm

plt.figure(figsize=(3, 3))
min_ability = np.min(b4.abilities)
max_ability = np.max(b4.abilities)

plt.rcParams["font.family"] = "Times New Roman"

sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize=(3, 3)) 
ax.grid(color='black', linestyle='--', linewidth=0.5)

ax.spines['bottom'].set_color('black')
ax.spines['left'].set_color('black')
ax.spines['right'].set_color('black')
ax.spines['top'].set_color('black')
ax.xaxis.label.set_color('black')
ax.yaxis.label.set_color('black')
ax.tick_params(axis='x', colors='black')
ax.tick_params(axis='y', colors='black')

# 使用 tab20 调色板
colors = cm.tab20.colors

for index in range(17):
    fairness_2 = ICC_function((1-abilities), b4.difficulties[index], b4.discriminations[index])
    plt.plot(abilities, fairness_2, label=f'{normalized_df.index[index]}', color=colors[index % len(colors)])

plt.fill_betweenx(np.arange(-0.05, 1.15, 0.1), (1-min_ability), (1-max_ability), color='gray', alpha=0.2)

plt.xlim(-0.05, 1.05)
plt.ylim(-0.05, 1.05)

plt.gca().set_aspect('equal', adjustable='box')

legend = plt.legend(loc='upper left', fontsize=9, bbox_to_anchor=(-1.05, 0.75), ncol=1, frameon=False)

for text in legend.get_texts():
    text.set_fontname('Times New Roman')
    text.set_color('black')

plt.xlabel(r'The inherent challenge of the dataset', fontsize=10, family='Times New Roman')
plt.ylabel(r'nDCG@10', fontsize=10, family='Times New Roman')

plt.grid(True)
plt.savefig("Figure4b.pdf", format="pdf", bbox_inches='tight')
plt.show()
